# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XsmiQTdKhuQ



## _Setup_ geral

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt

from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler, FunctionTransformer

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

#from IPython.core.pylabtools import figsize


#figsize(12, 8)

#sns.set()

In [3]:
countries = pd.read_csv("countries.csv")

In [4]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

In [5]:
countries["Region"].apply(lambda line: line.strip()).value_counts()

SUB-SAHARAN AFRICA      51
LATIN AMER. & CARIB     45
ASIA (EX. NEAR EAST)    28
WESTERN EUROPE          28
OCEANIA                 21
NEAR EAST               16
C.W. OF IND. STATES     12
EASTERN EUROPE          12
NORTHERN AFRICA          6
NORTHERN AMERICA         5
BALTICS                  3
Name: Region, dtype: int64

## Inicia sua análise a partir daqui

In [6]:
# Sua análise começa aqui.
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 20 columns):
Country             227 non-null object
Region              227 non-null object
Population          227 non-null int64
Area                227 non-null int64
Pop_density         227 non-null object
Coastline_ratio     227 non-null object
Net_migration       224 non-null object
Infant_mortality    224 non-null object
GDP                 226 non-null float64
Literacy            209 non-null object
Phones_per_1000     223 non-null object
Arable              225 non-null object
Crops               225 non-null object
Other               225 non-null object
Climate             205 non-null object
Birthrate           224 non-null object
Deathrate           223 non-null object
Agriculture         212 non-null object
Industry            211 non-null object
Service             212 non-null object
dtypes: float64(1), int64(2), object(17)
memory usage: 35.5+ KB


In [7]:
countries.isna().sum().sort_values(ascending=False)

Climate             22
Literacy            18
Industry            16
Service             15
Agriculture         15
Deathrate            4
Phones_per_1000      4
Birthrate            3
Infant_mortality     3
Net_migration        3
Other                2
Crops                2
Arable               2
GDP                  1
Coastline_ratio      0
Pop_density          0
Area                 0
Population           0
Region               0
Country              0
dtype: int64

In [8]:
countries.describe()

,Population,Area,GDP
count,2.270000e+02,2.270000e+02,226.000000
mean,2.874028e+07,5.982270e+05,9689.823009
std,1.178913e+08,1.790282e+06,10049.138513
min,7.026000e+03,2.000000e+00,500.000000
25%,4.376240e+05,4.647500e+03,1900.000000
50%,4.786994e+06,8.660000e+04,5550.000000
75%,1.749777e+07,4.418110e+05,15700.000000
max,1.313974e+09,1.707520e+07,55100.000000


In [9]:
# removendo espaços no inicio e fim da string
countries["Country"] = countries["Country"].apply(lambda line: line.strip())
countries["Region"] = countries["Region"].apply(lambda line: line.strip())

In [10]:
countries2 = countries.copy()

In [11]:
colunasNumericas = ["Pop_density", "Coastline_ratio", "Net_migration", "Infant_mortality","Literacy", 
                    "Phones_per_1000", "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", 
                    "Agriculture", "Industry", "Service"]

In [12]:
def toNumber(x):
    if pd.isna(x):
        return None
    
    x = x.strip()
    
    return float(x.replace(",", "."))

In [13]:
# colocando colunas nos seus tipos logicamente corretos

for column in colunasNumericas:
    countries2[column] = countries2[column].apply(toNumber) 

In [14]:
countries2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 20 columns):
Country             227 non-null object
Region              227 non-null object
Population          227 non-null int64
Area                227 non-null int64
Pop_density         227 non-null float64
Coastline_ratio     227 non-null float64
Net_migration       224 non-null float64
Infant_mortality    224 non-null float64
GDP                 226 non-null float64
Literacy            209 non-null float64
Phones_per_1000     223 non-null float64
Arable              225 non-null float64
Crops               225 non-null float64
Other               225 non-null float64
Climate             205 non-null float64
Birthrate           224 non-null float64
Deathrate           223 non-null float64
Agriculture         212 non-null float64
Industry            211 non-null float64
Service             212 non-null float64
dtypes: float64(16), int64(2), object(2)
memory usage: 35.5+ KB


In [15]:
countries2.describe()

,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
count,2.270000e+02,2.270000e+02,227.000000,227.000000,224.000000,224.000000,226.000000,209.000000,223.000000,225.000000,225.000000,225.000000,205.000000,224.000000,223.000000,212.000000,211.000000,212.000000
mean,2.874028e+07,5.982270e+05,379.047137,21.165330,0.038125,35.506964,9689.823009,82.838278,236.061435,13.797111,4.564222,81.638311,2.139024,22.114732,9.241345,0.150844,0.282711,0.565283
std,1.178913e+08,1.790282e+06,1660.185825,72.286863,4.889269,35.389899,10049.138513,19.722173,227.991829,13.040402,8.361470,16.140835,0.699397,11.176716,4.990026,0.146798,0.138272,0.165841
min,7.026000e+03,2.000000e+00,0.000000,0.000000,-20.990000,2.290000,500.000000,17.600000,0.200000,0.000000,0.000000,33.330000,1.000000,7.290000,2.290000,0.000000,0.020000,0.062000
25%,4.376240e+05,4.647500e+03,29.150000,0.100000,-0.927500,8.150000,1900.000000,70.600000,37.800000,3.220000,0.190000,71.650000,2.000000,12.672500,5.910000,0.037750,0.193000,0.429250
50%,4.786994e+06,8.660000e+04,78.800000,0.730000,0.000000,21.000000,5550.000000,92.500000,176.200000,10.420000,1.030000,85.700000,2.000000,18.790000,7.840000,0.099000,0.272000,0.571000
75%,1.749777e+07,4.418110e+05,190.150000,10.345000,0.997500,55.705000,15700.000000,98.000000,389.650000,20.000000,4.440000,95.440000,3.000000,29.820000,10.605000,0.221000,0.341000,0.678500
max,1.313974e+09,1.707520e+07,16271.500000,870.660000,23.060000,191.190000,55100.000000,100.000000,1035.600000,62.110000,50.680000,100.000000,4.000000,50.730000,29.740000,0.769000,0.906000,0.954000


In [16]:
# removendo valores missing
countries3 = countries2.copy() 
#countries2.dropna(axis = 0, inplace=True)

In [17]:
# verificando outliers
# for col in countries2.select_dtypes(["int", "float"]):
#     plt.figure(figsize=(10, 7))
#     plt.title(col)
#     plt.boxplot(countries2[col])
#     plt.show()

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [18]:
def q1():
    # Retorne aqui o resultado da questão 1.
    return list(countries2["Region"].value_counts().sort_index().index)

In [19]:
q1()

['ASIA (EX. NEAR EAST)',
 'BALTICS',
 'C.W. OF IND. STATES',
 'EASTERN EUROPE',
 'LATIN AMER. & CARIB',
 'NEAR EAST',
 'NORTHERN AFRICA',
 'NORTHERN AMERICA',
 'OCEANIA',
 'SUB-SAHARAN AFRICA',
 'WESTERN EUROPE']

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [20]:
def q2():
    # Retorne aqui o resultado da questão 2.
    
    # instanciando o objeto para discretizar
    discretizador = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile")
    
    # discretizando
    Pop_density_discretizado = discretizador.fit_transform(np.array(countries2["Pop_density"]).reshape(-1, 1))
    
    # obtendo 90º percentil
    percentil90 = np.quantile(Pop_density_discretizado, 0.9)
    
    num = len(Pop_density_discretizado[Pop_density_discretizado > percentil90])
    
    return int(num)

In [21]:
q2()

23

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [22]:
def q3():
    # Retorne aqui o resultado da questão 3.
    
    # objeto para converter categórico para numérico
    le = preprocessing.LabelEncoder()
    
    # objeto para aplicar oneHot
    enc = OneHotEncoder(handle_unknown='ignore')
    
    # variável temporária
    data = countries2[["Region", "Climate"]]
    
    data.fillna({"Climate": 0}, inplace=True)
    
    # transformando colunas region em numérica
    data["Region"] = le.fit_transform(data["Region"])
    
    # aplicando o oneHotEncoding
    values = enc.fit_transform(data).toarray()
    
    # Pegando os nomes das novas colunas
    colNames = list(enc.get_feature_names(["Region", "Climate"]))
    
    # criando o dataframe com os novos dados
    newData = pd.DataFrame(values, columns=colNames)
     
    # obtendo o número de novas colunas adicionadas
    return int(newData.shape[1])

In [23]:
q3()

/home/breno/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/breno/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


18

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [24]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [25]:
def q4():
    # Retorne aqui o resultado da questão 4.
    
    # definindo os passos do pipeline
    steps_pipeline = [('imputer', SimpleImputer(strategy='median')), 
                      ('scaler', StandardScaler())]

    # Obtendo as colunas para identificar as variáveis mais tarde
    columns = countries3.iloc[:, 2:].columns
    
    # Definindo o objeto pipeline
    pipeline = Pipeline(steps=steps_pipeline)
    
    # 'treinando' o pipeline
    pipeline.fit(countries3.iloc[:, 2:])
    
    arable = pd.DataFrame(pipeline.transform(np.array(test_country[2:]).reshape(1, -1)), columns=columns).loc[:, "Arable"][0]
    
    return float(round(arable, 3))

In [26]:
q4()

-1.047

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [27]:
def q5():
    # Retorne aqui o resultado da questão 4.
    migracao = countries2["Net_migration"]
    threshold = 0.7
    
    q1 = migracao.quantile(0.25)
    q3 = migracao.quantile(0.75)
    iqr = q3 - q1
    
    interval = [q1 - 1.5*iqr, q3 + 1.5*iqr]
    
    outliers_abaixo = migracao[migracao < interval[0]]
    outliers_acima = migracao[migracao > interval[1]]
    remove = bool((len(outliers_abaixo)+len(outliers_acima)/len(migracao) < threshold))
    
    return (len(outliers_abaixo), len(outliers_acima), remove)

In [28]:
q5()

(24, 26, False)

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [29]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']

newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [30]:
count_vectorizer = CountVectorizer()

newsgroups_counts = count_vectorizer.fit_transform(newsgroup.data)

In [31]:
def q6():
    # Retorne aqui o resultado da questão 4.
    return int(newsgroups_counts[:, count_vectorizer.vocabulary_["phone"]].toarray().sum())

In [32]:
q6()

213

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [33]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(newsgroup.data)

newsgroups_tfidf_vectorized = tfidf_vectorizer.transform(newsgroup.data)

In [34]:
def q7():
    # Retorne aqui o resultado da questão 4.
    idf_value = newsgroups_tfidf_vectorized[:, tfidf_vectorizer.vocabulary_["phone"]].toarray().sum()
    
    return float(round(idf_value, 3))

In [35]:
q7()

8.888

FIM